# TensorFlow Datasets

A biblioteca TensorFlow Datasets (TFDS) fornece uma coleção de datasets prontos que podem ser usados com TensorFlow, Jax e outros frameworks de aprendizado de máquina.

Ela cuida do download e da preparação dos dados de forma determinística e da construção de um `tf.data.Dataset` (ou `np.array`).

Observação: Não confunda [TFDS](https://www.tensorflow.org/datasets) (esta biblioteca) com `tf.data` (API TensorFlow para construir pipelines de dados eficientes). TFDS é um wrapper de alto nível em torno de `tf.data`. Se você não estiver familiarizado com esta API, recomendamos que você leia primeiro o [guia oficial do tf.data](https://www.tensorflow.org/guide/data).


Copyright 2020 The TensorFlow Datasets Authors, Licensed under the Apache License, Version 2.0

<table class="tfo-notebook-buttons" align="left">
  <td>     <a target="_blank" href="https://www.tensorflow.org/datasets/overview"><img src="https://www.tensorflow.org/images/tf_logo_32px.png">Ver em TensorFlow.org</a>
</td>
  <td>     <a target="_blank" href="https://colab.research.google.com/github/tensorflow/docs-l10n/blob/master/site/pt-br/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/colab_logo_32px.png">Executar no Google Colab</a>
</td>
  <td>     <a target="_blank" href="https://github.com/tensorflow/docs-l10n/blob/master/site/pt-br/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/GitHub-Mark-32px.png">Ver fonte no GitHub</a>
</td>
  <td>     <a href="https://storage.googleapis.com/tensorflow_docs/docs-l10n/site/pt-br/datasets/overview.ipynb"><img src="https://www.tensorflow.org/images/download_logo_32px.png">Baixar notebook</a>
</td>
</table>

## Instalação

A biblioteca TFDS existe em dois pacotes:

- `pip install tensorflow-datasets`: a versão estável, lançada a cada poucos meses.
- `pip install tfds-nightly`: lançada todos os dias, contém as últimas versões dos datasets.

Este colab usa `tfds-nightly`:


In [ ]:
!pip install -q tfds-nightly tensorflow matplotlib

In [ ]:
import matplotlib.pyplot as plt
import numpy as np
import tensorflow as tf

import tensorflow_datasets as tfds

## Encontre datasets disponíveis

Todos os construtores de datasets são subclasses de `tfds.core.DatasetBuilder`. Para obter a lista de construtores disponíveis, use `tfds.list_builders()` ou consulte nosso [catálogo](https://www.tensorflow.org/datasets/catalog/overview).

In [ ]:
tfds.list_builders()

## Carregue um dataset

### tfds.load

A maneira mais fácil de carregar um datasets é usando `tfds.load`. Ele vai:

1. Baixar os dados e salvá-os como arquivos [`tfrecord`](https://www.tensorflow.org/tutorials/load_data/tfrecord).
2. Carregar o `tfrecord` e criar o `tf.data.Dataset`.


In [ ]:
ds = tfds.load('mnist', split='train', shuffle_files=True)
assert isinstance(ds, tf.data.Dataset)
print(ds)

Alguns argumentos comuns:

- `split=`: qual divisão ler (por exemplo, `'train'`, `['train', 'test']`, `'train[80%:]'`,...). Consulte o [guia da API split](https://www.tensorflow.org/datasets/splits).
- `shuffle_files=`: controla se os arquivos devem ser embaralhados entre cada época (o TFDS armazena grandes datasets em múltiplos arquivos menores).
- `data_dir=`: local onde o dataset é salvo (o padrão é `~/tensorflow_datasets/`)
- `with_info=True`: retorna `tfds.core.DatasetInfo` contendo metadados do dataset
- `download=False`: desativa o download


### tfds.builder

`tfds.load` é um wrapper estreito em torno de `tfds.core.DatasetBuilder`. Você pode obter a mesma saída usando a API `tfds.core.DatasetBuilder`:

In [ ]:
builder = tfds.builder('mnist')
# 1. Create the tfrecord files (no-op if already exists)
builder.download_and_prepare()
# 2. Load the `tf.data.Dataset`
ds = builder.as_dataset(split='train', shuffle_files=True)
print(ds)

### `tfds build` CLI

Se quiser gerar um dataset específico, você pode usar a [linha de comando `tfds`](https://www.tensorflow.org/datasets/cli). Por exemplo:

```sh
tfds build mnist
```

Consulte [a documentação](https://www.tensorflow.org/datasets/cli) para conhecer os sinalizadores disponíveis.

## Itere sobre um dataset

### Como um dict

Por padrão, o objeto `tf.data.Dataset` contém um `dict` de objetos `tf.Tensor`:

In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.take(1)  # Only take a single example

for example in ds:  # example is `{'image': tf.Tensor, 'label': tf.Tensor}`
  print(list(example.keys()))
  image = example["image"]
  label = example["label"]
  print(image.shape, label)

Para descobrir os nomes das chaves de um `dict` e sua estrutura, consulte a documentação do dataset no [nosso catálogo](https://www.tensorflow.org/datasets/catalog/overview#all_datasets). Por exemplo: a [documentação do mnist](https://www.tensorflow.org/datasets/catalog/mnist).

### Como tupla (`as_supervised=True`)

Ao usar `as_supervised=True`, você pode obter uma tupla `(features, label)` para datasets supervisionados.

In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in ds:  # example is (image, label)
  print(image.shape, label)

### Como numpy (`tfds.as_numpy`)

Usa `tfds.as_numpy` para converter:

- `tf.Tensor` -&gt; `np.array`
- `tf.data.Dataset` -&gt; `Iterator[Tree[np.array]]` (`Tree` pode ser `Dict` aninhada arbitrariamente, `Tuple`)


In [ ]:
ds = tfds.load('mnist', split='train', as_supervised=True)
ds = ds.take(1)

for image, label in tfds.as_numpy(ds):
  print(type(image), type(label), label)

### Como tf.Tensor em lote (`batch_size=-1`)

Usando `batch_size=-1`, você pode carregar o dataset completo num único lote.

Isto pode ser combinado com `as_supervised=True` e `tfds.as_numpy` para obter os dados como `(np.array, np.array)`:


In [ ]:
image, label = tfds.as_numpy(tfds.load(
    'mnist',
    split='test',
    batch_size=-1,
    as_supervised=True,
))

print(type(image), image.shape)

Tenha cuidado para que seu dataset caiba na memória e para que todos os exemplos tenham o mesmo formato.

## Faça benchmark dos seus datasets

O benchmarking de um dataset é uma chamada simples `tfds.benchmark` em qualquer iterável (por exemplo, `tf.data.Dataset`, `tfds.as_numpy`,...).


In [ ]:
ds = tfds.load('mnist', split='train')
ds = ds.batch(32).prefetch(1)

tfds.benchmark(ds, batch_size=32)
tfds.benchmark(ds, batch_size=32)  # Second epoch much faster due to auto-caching

- Não esqueça de normalizar os resultados por tamanho de lote com o kwarg `batch_size=`.
- No resumo, o primeiro lote de aquecimento é separado dos demais para capturar o tempo extra de configuração `tf.data.Dataset` (por exemplo, inicialização de buffers,...).
- Observe como a segunda iteração é muito mais rápida devido ao [cache automático do TFDS](https://www.tensorflow.org/datasets/performances#auto-caching).
- O `tfds.benchmark` retorna um `tfds.core.BenchmarkResult` que pode ser inspecionado para análise posterior.

### Crie um pipeline de ponta a ponta

Para ir mais longe, você pode explorar:

- Nosso [exemplo Keras ponta a ponta](https://www.tensorflow.org/datasets/keras_example) para ver um pipeline de treinamento completo (com lote, embaralhamento,...).
- Nosso [guia de desempenho](https://www.tensorflow.org/datasets/performances) para melhorar a velocidade de seus pipelines (dica: use `tfds.benchmark(ds)` para fazer benchmark dos seus datasets).


## Visualização

### tfds.as_dataframe

Os objetos `tf.data.Dataset` podem ser convertidos em [`pandas.DataFrame`](https://pandas.pydata.org/pandas-docs/stable/reference/api/pandas.DataFrame.html) com `tfds.as_dataframe` para serem visualizados no [Colab](https://colab.research.google.com).

- Adicione o `tfds.core.DatasetInfo` como segundo argumento de `tfds.as_dataframe` para visualizar imagens, áudio, textos, vídeos,...
- Use `ds.take(x)` para exibir apenas os primeiros `x` exemplos. `pandas.DataFrame` carregará o dataset completo na memória o que pode exigir muitos recursos para exibir.

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

tfds.as_dataframe(ds.take(4), info)

### tfds.show_examples

`tfds.show_examples` retorna um `matplotlib.figure.Figure` (no momento, somente datasets de imagem são suportados):

In [ ]:
ds, info = tfds.load('mnist', split='train', with_info=True)

fig = tfds.show_examples(ds, info)

## Acesse os metadados do dataset

Todos os construtores incluem um objeto `tfds.core.DatasetInfo` contendo os metadados do dataset.

Ele pode ser acessado através das seguintes APIs:

- `tfds.load`:


In [ ]:
ds, info = tfds.load('mnist', with_info=True)

- `tfds.core.DatasetBuilder`:

In [ ]:
builder = tfds.builder('mnist')
info = builder.info

O dataset info contêm informações adicionais sobre o dataset (versão, citação, página inicial, descrição,...).

In [ ]:
print(info)

### Metadados de características (nomes de rótulos, formato de imagem,...)

Acesse `tfds.features.FeatureDict`:

In [ ]:
info.features

Número de classes, nomes de rótulos:

In [ ]:
print(info.features["label"].num_classes)
print(info.features["label"].names)
print(info.features["label"].int2str(7))  # Human readable version (8 -> 'cat')
print(info.features["label"].str2int('7'))

Formatos, dtypes:

In [ ]:
print(info.features.shape)
print(info.features.dtype)
print(info.features['image'].shape)
print(info.features['image'].dtype)

### Metadados de divisão (por exemplo, nomes de splits, número de exemplos,...)

Acesse `tfds.core.SplitDict`:

In [ ]:
print(info.splits)

Divisões (splits) disponíveis:

In [ ]:
print(list(info.splits.keys()))

Obtenha informações sobre a divisão individual:

In [ ]:
print(info.splits['train'].num_examples)
print(info.splits['train'].filenames)
print(info.splits['train'].num_shards)

Também funciona com a API subsplit:

In [ ]:
print(info.splits['train[15%:75%]'].num_examples)
print(info.splits['train[15%:75%]'].file_instructions)

## Solução de problemas

### Download manual (se o download falhar)

Se o download falhar por algum motivo (por exemplo, offline,...). Você sempre pode baixar manualmente os dados e colocá-los no `manual_dir` (o padrão é `~/tensorflow_datasets/downloads/manual/`.

Para descobrir quais URLs baixar, procure:

- Para novos datasets (implementados como um diretório): [`tensorflow_datasets/`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/) `<type>/<dataset_name>/checksums.tsv`. Por exemplo: [`tensorflow_datasets/datasets/bool_q/checksums.tsv`](https://github.com/tensorflow/datasets/blob/master/tensorflow_datasets/datasets/bool_q/checksums.tsv).

    Você pode encontrar o local da fonte do dataset no [nosso catálogo](https://www.tensorflow.org/datasets/catalog/overview).

- Para datasets antigos: [`tensorflow_datasets/url_checksums/<dataset_name>.txt`](https://github.com/tensorflow/datasets/tree/master/tensorflow_datasets/url_checksums)

### Corrigindo o `NonMatchingChecksumError`

A biblioteca TFDS garante o determinismo validandoos checksums das URLs baixadas. Se um `NonMatchingChecksumError` for lançado, isto pode indicar que:

- O site pode estar fora do ar (por exemplo, `503 status code`). Por favor, verifique a URL.
- Para URLs do Google Drive, tente novamente mais tarde, pois o Drive às vezes rejeita downloads quando muitas pessoas acessam a mesmo URL. Veja [bug](https://github.com/tensorflow/datasets/issues/1482)
- Os arquivos dos datasets originais podem ter sido alterados. Neste caso, o construtor do conjunto de dados TFDS deve ser atualizado. Abra um novo issue no GitHub ou envie um pull request:
    - Registre os novos checksums com `tfds build --register_checksums`
    - Em algum momento, atualize o código de geração do dataset.
    - Atualize a `VERSION` do dataset
    - Atualize as `RELEASE_NOTES` do dataset: o que causou a alteração dos checksums? Alguns exemplos mudaram?
    - Garanta que o dataset ainda possa ser compilado.
    - Envie-nos um pull request

Observação: você também pode inspecionar o arquivo baixado em `~/tensorflow_datasets/download/`.


## Citação

Se você estiver usando o `tensorflow-datasets` num artigo, inclua a citação a seguir, além de qualquer citação específica dos datasets usados ​​(que pode ser encontrada no [catálogo de datasets](https://www.tensorflow.org/datasets/catalog/overview)).

```
@misc{TFDS,
  title = { {TensorFlow Datasets}, A collection of ready-to-use datasets},
  howpublished = {\url{https://www.tensorflow.org/datasets}},
}
```